In [141]:
!pip install xgboost




In [157]:

#In this project, you are tasked to build a model(s) that predict a player's overall rating given the player's profile.
#Demonstrate the data preparation & feature extraction process
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [158]:
import pandas as pd
import numpy as np
file = "male_players (legacy).csv"
file22="players_22.csv"
data22 = pd.read_csv(file22)
# Read the CSV file
data = pd.read_csv(file)


C:\Users\hp\AppData\Local\Temp\ipykernel_12796\1990246387.py:5: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data22 = pd.read_csv(file22)
C:\Users\hp\AppData\Local\Temp\ipykernel_12796\1990246387.py:7: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file)


In [159]:
# Step 2: Drop specified columns
columns_to_drop = [
    'player_id', 'dob', 'player_tags', 'club_contract_valid_until_year', 'player_url', 'club_jersey_number', 'club_name', 'fifa_update',
    'long_name', 'short_name', 'league_id', 'player_face_url', 'nationality_id', 'preferred_foot', 'club_contract_valid_until_year',
    'fifa_update_date', 'club_position', 'league_name', 'club_team_id', 'nation_team_id', 'player_traits', 'club_joined_date', 'league_level',
    'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm', 'lb', 'cb', 'nationality_name', 'real_face',
    'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lwb', 'rwb', 'lcb', 'rcb', 'rb', 'gk', 'player_face_url', 'ls', 'st', 'body_type', 'fifa_version'
]
columns_to_drop22 = [
    'sofifa_id', 'dob', 'player_tags', 'club_contract_valid_until', 'player_url', 'club_jersey_number', 'club_name','club_loaned_from','nation_position',
    'long_name', 'short_name', 'player_face_url', 'nationality_id', 'preferred_foot','club_position', 'league_name', 'club_team_id', 'nation_team_id', 'player_traits', 'club_joined', 'league_level',
    'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm', 'lb', 'cb', 'nationality_name', 'real_face','nation_logo_url','nation_flag_url',
    'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lwb', 'rwb', 'lcb', 'rcb', 'rb', 'gk', 'player_face_url', 'ls', 'st', 'body_type','club_logo_url','club_flag_url'
]
data = data.drop(columns=columns_to_drop, axis=1)
data22=data22.drop(columns=columns_to_drop22, axis=1)

# Step 3: Drop columns with 30% or more null values
biased_threshold = 0.30 * len(data)
data = data.loc[:, data.isna().sum() < biased_threshold]
data22=data22.loc[:, data22.isna().sum() < biased_threshold]
data

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,CF,93,95,100500000.0,550000.0,27,169,67,3,4,...,76,NaN,25,21,20,6,11,15,14,8
1,"LW, LM",92,92,79000000.0,375000.0,29,185,80,4,5,...,85,NaN,22,31,23,7,11,15,14,11
2,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,2,4,...,80,NaN,29,26,26,10,8,11,5,15
3,ST,90,90,52500000.0,275000.0,32,195,95,4,4,...,91,NaN,25,41,27,13,15,10,9,12
4,GK,90,90,63500000.0,300000.0,28,193,92,4,1,...,37,NaN,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,RW,46,61,110000.0,700.0,18,180,73,3,2,...,54,40.0,23,21,25,9,13,13,12,7
161579,LB,46,58,110000.0,750.0,19,188,83,3,2,...,31,35.0,50,51,45,6,14,8,13,14
161580,CDM,46,58,110000.0,500.0,19,181,73,2,2,...,37,35.0,36,45,50,8,9,7,14,9
161581,"RW, LW",46,70,150000.0,500.0,17,175,68,3,2,...,63,43.0,19,17,14,13,12,14,7,13


In [161]:
# Function to check for mixed data types in a column
def check_mixed_types(column):
    types = column.apply(lambda x: type(x)).unique()
    return len(types) > 1

#  Separate numeric data from non-numeric data
numeric_data = data.select_dtypes(include=np.number)
numeric_data22=data22.select_dtypes(include=np.number)
non_numeric_data = data.select_dtypes(include=['object'])
non_numeric_data22 = data22.select_dtypes(include=['object'])

#  Impute missing values for numeric data with mean
numeric_data = numeric_data.apply(lambda col: col.fillna(col.mean()))
numeric_data22 = numeric_data22.apply(lambda col: col.fillna(col.mean()))

# Impute missing values for non-numeric data with mode
for column in non_numeric_data.columns:
    mode = non_numeric_data[column].mode()[0]
    non_numeric_data[column].fillna(mode, inplace=True)

# Impute missing values for non-numeric22 data with mode
for column in non_numeric_data22.columns:
    mode = non_numeric_data22[column].mode()[0]
    non_numeric_data22[column].fillna(mode, inplace=True)
















,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,international_reputation,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93,95,100500000.0,550000.0,27,169,67,3,4,5,...,76,57.816892,25,21,20,6,11,15,14,8
1,92,92,79000000.0,375000.0,29,185,80,4,5,5,...,85,57.816892,22,31,23,7,11,15,14,11
2,90,90,54500000.0,275000.0,30,180,80,2,4,5,...,80,57.816892,29,26,26,10,8,11,5,15
3,90,90,52500000.0,275000.0,32,195,95,4,4,5,...,91,57.816892,25,41,27,13,15,10,9,12
4,90,90,63500000.0,300000.0,28,193,92,4,1,5,...,37,57.816892,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46,61,110000.0,700.0,18,180,73,3,2,1,...,54,40.000000,23,21,25,9,13,13,12,7
161579,46,58,110000.0,750.0,19,188,83,3,2,1,...,31,35.000000,50,51,45,6,14,8,13,14
161580,46,58,110000.0,500.0,19,181,73,2,2,1,...,37,35.000000,36,45,50,8,9,7,14,9
161581,46,70,150000.0,500.0,17,175,68,3,2,1,...,63,43.000000,19,17,14,13,12,14,7,13


In [163]:
# Encode categorical columns using LabelEncoder
label_encoders = {}
for column in non_numeric_data.columns:
    label_encoders[column] = LabelEncoder()
    non_numeric_data[column] = label_encoders[column].fit_transform(non_numeric_data[column])

# Encode categorical columns for test data using the same LabelEncoders
for column in non_numeric_data22.columns:
    if column in label_encoders:
        non_numeric_data22[column] = label_encoders[column].transform(non_numeric_data22[column])
    else:
        # Handle cases where the test data might have columns not present in the training data
        print(f"Warning: Column {column} not found in training data encoders and will be ignored.")
        non_numeric_data22 = non_numeric_data22.drop(columns=[column])
    # Combine numeric data and label encoded data
processed_data = pd.concat([numeric_data, non_numeric_data], axis=1)
data22=pd.concat([numeric_data22, non_numeric_data22], axis=1)
corr_matrix=processed_data.corr()
data22

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,nation_jersey_number,weak_foot,skill_moves,...,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,player_positions,work_rate
0,93,93,78000000.0,320000.0,34,170,72,10.000000,4,4,...,35,24,6,11,15,14,8,36.439962,1762,7
1,92,92,119500000.0,270000.0,32,185,81,9.000000,4,4,...,42,19,15,6,12,8,10,36.439962,1838,2
2,91,91,45000000.0,270000.0,36,187,83,7.000000,4,5,...,32,24,7,11,15,14,11,36.439962,1918,1
3,91,91,129000000.0,270000.0,29,175,68,12.567852,5,5,...,32,29,9,9,15,15,11,36.439962,1090,2
4,91,91,125500000.0,350000.0,30,181,70,7.000000,5,4,...,65,53,15,13,5,10,13,36.439962,494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,180,64,12.567852,3,2,...,43,48,6,10,5,15,13,36.439962,289,8
19235,47,59,110000.0,500.0,19,175,70,12.567852,3,2,...,44,47,11,12,6,8,10,36.439962,493,8
19236,47,55,100000.0,500.0,21,178,72,12.567852,3,2,...,44,48,8,6,7,10,6,36.439962,493,8
19237,47,60,110000.0,500.0,19,173,66,12.567852,3,2,...,14,11,7,10,7,14,15,36.439962,1838,8


In [147]:
definers = []
for col in processed_data.columns:
    if col != 'overall':
        if (corr_matrix.loc['overall', col] > 0.45) or (corr_matrix.loc['overall', col] < -0.45):
            definers.append(col)
correlation_dict = {col: corr_matrix.loc['overall', col] for col in definers}

# Sort the definers list based on the correlation values with 'overall' in descending order
sorted_definers = sorted(definers, key=lambda x: correlation_dict[x], reverse=True)
sorted_definers

['movement_reactions',
 'potential',
 'passing',
 'wage_eur',
 'mentality_composure',
 'value_eur',
 'dribbling',
 'attacking_short_passing',
 'mentality_vision',
 'international_reputation',
 'skill_long_passing',
 'power_shot_power',
 'physic',
 'age',
 'skill_ball_control']

In [165]:
processed_data


,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,weak_foot,skill_moves,international_reputation,...,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,player_positions,work_rate
0,93,95,100500000.0,550000.0,27,169,67,3,4,5,...,25,21,20,6,11,15,14,8,412,7
1,92,92,79000000.0,375000.0,29,185,80,4,5,5,...,22,31,23,7,11,15,14,11,1128,1
2,90,90,54500000.0,275000.0,30,180,80,2,4,5,...,29,26,26,10,8,11,5,15,1541,1
3,90,90,52500000.0,275000.0,32,195,95,4,4,5,...,25,41,27,13,15,10,9,12,1838,7
4,90,90,63500000.0,300000.0,28,193,92,4,1,5,...,25,25,25,87,85,92,90,86,719,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46,61,110000.0,700.0,18,180,73,3,2,1,...,23,21,25,9,13,13,12,7,1643,8
161579,46,58,110000.0,750.0,19,188,83,3,2,1,...,50,51,45,6,14,8,13,14,721,8
161580,46,58,110000.0,500.0,19,181,73,2,2,1,...,36,45,50,8,9,7,14,9,289,2
161581,46,70,150000.0,500.0,17,175,68,3,2,1,...,19,17,14,13,12,14,7,13,1694,8


In [171]:
# scaling te data after label encoding
overall=processed_data['overall']
processed_data=processed_data[sorted_definers]
scaler=StandardScaler()
scaler_2=scaler.fit_transform(processed_data)

processed_data=pd.DataFrame(scaler_2,columns=processed_data.columns)


KeyError: 'overall'

TRAINING AND TESTING

In [169]:
# splitting data for train and test.
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(processed_data,test_size=0.2, random_state=42)  # splitt into two sets using 20 % 80%
print(len(train_set), "train +", len(test_set), "test")  # splitting datset using 20 % 80%


129266 train + 32317 test


In [111]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X = processed_data.copy()
y=overall

In [112]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# getting best features
forest=RandomForestRegressor(n_estimators=250,n_jobs=-1)
forest=forest.fit(Xtrain, Ytrain)
forest_predictions = forest.predict(Xtest)
forest_score=forest.score(Xtest,Ytest)
forest_rmse = np.sqrt(mean_squared_error(Ytest, forest_predictions))

print(f'RandomForestRegressor RMSE: {forest_rmse}')

print(f'test score: {forest_score}')




RandomForestRegressor RMSE: 0.8363558688741984
test score: 0.9858869672255272


In [113]:
# Initialize and fit GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=250, random_state=42)
gbr=gbr.fit(Xtrain, Ytrain)
gbr_predictions = gbr.predict(Xtest)
gbr_rmse = np.sqrt(mean_squared_error(Ytest, gbr_predictions))
gbr_score=gbr.score(Xtest,Ytest)
print(f'GradientBoostingRegressor RMSE: {gbr_rmse}')
print(f'test_score: {gbr_score}')


GradientBoostingRegressor RMSE: 1.069101790872209
test_score: 0.9769391003150766


In [114]:
xgb = XGBRegressor(n_estimators=250, random_state=42, n_jobs=-1)
xgb.fit(Xtrain, Ytrain)
xgb_predictions = xgb.predict(Xtest)
xgb_rmse = np.sqrt(mean_squared_error(Ytest, xgb_predictions))
xgb_score=xgb.score(Xtest,Ytest)
print(f'XGBoostRegressor RMSE: {xgb_rmse}')
print(f'test_score: {xgb_score}')


XGBoostRegressor RMSE: 0.8857200425956522
test_score: 0.9841718164802625


UTILIZING FEATURE IMPORTANCE

In [115]:
# Get feature importances for the forest regressor because it was the best model
feature_importances = forest.feature_importances_

# Get feature names
feature_names = Xtrain.columns

# Create a dictionary of feature names and their importances
feature_importance_dict = dict(zip(feature_names, feature_importances))

# Sort features by importance (from highest to lowest)
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Select the top 7 features based on importance
selected_features = sorted_features[:7]

# Extract the names of the selected features
selected_feature_names = [feature[0] for feature in selected_features]


TESTING THE NEW IMPORTANT FEATURES

In [116]:
# Utilize the selected features
X_train_selected = Xtrain[selected_feature_names]
X_test_selected = Xtest[selected_feature_names]

# Initializing and train a new model with selected features
forest = RandomForestRegressor(n_estimators=250, random_state=42)
forest.fit(X_train_selected, Ytrain)

# Make predictions with the new model
y_pred_selected = forest.predict(X_test_selected)

# Evaluate the new model's performance
from sklearn.metrics import mean_squared_error
rmse_selected = np.sqrt(mean_squared_error(Ytest, y_pred_selected))
print(rmse_selected)

0.9089292902568412


CROSS-VALIDATION AND FINETUNING


In [117]:
#selectin random forest because it gave te least rmse and highest test score.
rf_model = {
    'name': 'RandomForestRegressor',
    'model': RandomForestRegressor(),
    'params': {
        'n_estimators': [50, 100, 150],
        'max_depth': [ 10, 20,30],
    }
}


In [122]:
#saving a scaler model for the flask deployment
import pickle
model=StandardScaler()
filename=r'C:\Users\hp\Documents\TEXT BOOKS\year 3_sem2\jupyternotebooks\scaler.pkl' 
with open(filename, 'wb') as file:
    pickle.dump(model,file)

In [123]:
from sklearn.metrics import make_scorer, mean_squared_error

In [124]:


# Function to perform GridSearchCV and return best model and results
def perform_grid_search(model_config, Xtrain, Ytrain):
    grid_search = GridSearchCV(model_config['model'], model_config['params'], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(Xtrain, Ytrain)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    best_estimator = grid_search.best_estimator_

    return best_params, best_score, best_estimator

# Assuming Xtrain and Ytrain are already defined

# Perform GridSearchCV for RandomForestRegressor
best_params, best_score, best_rf_model = perform_grid_search(rf_model, Xtrain, Ytrain)

# Print the results
print(f"Results for {rf_model['name']}:\n")
print(f"Best Parameters: {best_params}")
print(f"Best  MSE: {best_score}")
print("=============================================")

Results for RandomForestRegressor:

Best Parameters: {'max_depth': 30, 'n_estimators': 150}
Best  MSE: -0.732413300240451


SAVING THE MODEL


In [125]:
import pickle


filename=r'C:\Users\hp\Documents\TEXT BOOKS\year 3_sem2\jupyternotebooks\best_rf_model.pkl' 
with open(filename, 'wb') as file:
    pickle.dump(best_rf_model,file)

In [170]:
overall=data22['overall']
y_22=overall
X_22=data22.drop(['overall'])
scaler=StandardScaler()
X_22=scaler.fit_transform(X_22)
X_22


KeyError: '[3.9575014466497884, 3.8121537226825635, 3.6668059987153385, 3.6668059987153385, 3.6668059987153385, 3.6668059987153385, 3.6668059987153385, 3.5214582747481136, 3.5214582747481136, 3.5214582747481136, 3.5214582747481136, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.3761105507808886, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.2307628268136637, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 3.0854151028464387, 2.9400673788792138, 2.9400673788792138, 2.940067

In [167]:
# load the model from disk to test
best_rf_model = pickle.load(open(filename, 'rb'))
y_new_pred = best_rf_model.predict(X_22[sorted_definers])




EVALUATING MODEL PERFORMANCE ON NEW DATASET

In [130]:
new_mse = mean_squared_error(y_22[:-47], y_new_pred)
print(f'Mean Squared Error on New Data: {new_mse}')      # te code is wrong


Mean Squared Error on New Data: 763.9867178731876
